# Fake Page View generator

This notebook generates page views based on a three-hour dataset.

In [1]:
%%bash
python3 -m pip install kafka-python

In [2]:
# Initialize Kafka Topics
from kafka import KafkaClient
from kafka.admin import KafkaAdminClient, NewTopic

TOPICS = ["clicks", "clicks-cleaned", "clicks-calculated", "clicks-calculated-ddos", "clicks-calculated-forbidden"]

kafka_client = KafkaClient("localhost:9092")
existing_topics = kafka_client.topic_partitions

admin_client = KafkaAdminClient(bootstrap_servers="localhost:9092")
topic_list = []
for topicname in TOPICS:
    if topicname not in existing_topics:
        print("Creating topic: {}".format(topicname))
        topic_list.append(NewTopic(name=topicname, num_partitions=1, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

Creating topic: clicks
Creating topic: clicks-cleaned
Creating topic: clicks-calculated
Creating topic: clicks-calculated-ddos
Creating topic: clicks-calculated-forbidden


CreateTopicsResponse_v0(topic_errors=[(topic='clicks', error_code=0), (topic='clicks-calculated', error_code=0), (topic='clicks-cleaned', error_code=0), (topic='clicks-calculated-ddos', error_code=0), (topic='clicks-calculated-forbidden', error_code=0)])

In [3]:
# Extract archive of click data
import zipfile

ARCHIVEF = "20180113-6-9-with-nulls.json.zip"
CLICKSF = "20180113-6-9-with-nulls.json"

with zipfile.ZipFile(ARCHIVEF,"r") as zip_ref:
    zip_ref.extractall(".")

In [4]:
# SENDMODE = "kafka"
SENDMODE = "http"

In [6]:
# Send click data
import json
import time
from datetime import datetime

import requests
from kafka import KafkaProducer
from kafka.errors import KafkaError
from IPython.display import clear_output

producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

orig_start = 0
replay_start = time.time() * 1000

counter = 0
i = 0

print("Simulating website traffic...")

with open(CLICKSF, 'r') as f:
    first = True
    start_ts = 0
    for line in f:
        click = json.loads(line)

        orig_event = click["ts_ingest"] or 0
        
        if orig_event:
            curr_ts = time.time() * 1000

            if first:
                first = False
                orig_start = click["ts_ingest"]

            difference = (orig_event - orig_start) - (curr_ts - replay_start)

            if difference > 0:
                print("sleeping {0:.2f} seconds".format(difference/1000.0))
                time.sleep(difference/1000.0)
            
        click['ts_ingest'] = round((orig_event - orig_start) + replay_start)

        if SENDMODE == "kafka":            
            future = producer.send('clicks', click)
            try:
                record_metadata = future.get(timeout=10)
            except KafkaError:
                log.exception()
                pass
        else:
            #print(f"sending {line}")
            requests.post("http://localhost:5000/clicks", data=json.dumps(click).encode("utf-8"), headers={'Content-Type':'application/json'})
        
        print("✓ {}".format(datetime.fromtimestamp(orig_event/1000.0)))
        if i > 1000:
            clear_output()
            i = 0
        i = i+1


✓ 2018-01-13 07:01:52.850000
✓ 2018-01-13 07:01:52.851000
✓ 2018-01-13 07:01:52.856000
✓ 2018-01-13 07:01:52.860000
✓ 1970-01-01 00:00:00
✓ 2018-01-13 07:01:52.861000
✓ 2018-01-13 07:01:52.868000
✓ 2018-01-13 07:01:52.874000
✓ 2018-01-13 07:01:52.888000
✓ 2018-01-13 07:01:52.888000
✓ 2018-01-13 07:01:52.899000
✓ 2018-01-13 07:01:52.906000
✓ 2018-01-13 07:01:52.919000
✓ 2018-01-13 07:01:52.920000
✓ 2018-01-13 07:01:52.923000
✓ 2018-01-13 07:01:52.927000
✓ 2018-01-13 07:01:52.935000
✓ 2018-01-13 07:01:52.938000
✓ 2018-01-13 07:01:52.947000
✓ 2018-01-13 07:01:52.954000
✓ 1970-01-01 00:00:00
✓ 2018-01-13 07:01:52.955000
✓ 2018-01-13 07:01:52.958000
✓ 2018-01-13 07:01:52.959000
✓ 2018-01-13 07:01:52.965000
✓ 2018-01-13 07:01:52.967000
✓ 2018-01-13 07:01:52.971000
✓ 2018-01-13 07:01:52.975000
✓ 2018-01-13 07:01:52.981000
✓ 2018-01-13 07:01:52.988000
✓ 2018-01-13 07:01:52.988000
✓ 2018-01-13 07:01:52.995000
✓ 2018-01-13 07:01:52.997000
✓ 2018-01-13 07:01:53
✓ 2018-01-13 07:01:53.014000
✓ 2018

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /clicks (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9430096da0>: Failed to establish a new connection: [Errno 111] Connection refused'))